In [582]:
from PIL import Image
import pytesseract
import argparse
import cv2
import os
import numpy as np
import random as rng
rng.seed(12345)

In [9]:
# image = cv2.imread("testset/22.jpg")
# gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# # check to see if we should apply thresholding to preprocess the
# # image
# #     if "thresh" == "thresh":
# gray = cv2.threshold(gray, 127, 255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)[1]

In [10]:
# from matplotlib import pyplot as plt
# plt.imshow(gray, interpolation='nearest')
# plt.show()

In [11]:
# # load the image as a PIL/Pillow image, apply OCR, and then delete
# # the temporary file
# text = pytesseract.image_to_string(gray)
# # os.remove(filename)
# print(text)
 
# # # show the output images
# # cv2.imshow("Image", image)
# # cv2.imshow("Output", gray)
# # cv2.waitKey(0)

In [276]:
import re
p1 = r"(.*[0-9])"
pattern1 = re.compile(p1)
def readImgOrg(path):
    image = cv2.imread(path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
 
    # check to see if we should apply thresholding to preprocess the
    # image
#     if "thresh" == "thresh":
    gray1 = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY,11,2)
#     gray2 = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)[1]
    gray = gray1
    # noise
#     elif "blur" == "blur":
#     gray = cv2.medianBlur(gray, 3)
    # write the grayscale image to disk as a temporary file so we can
    # apply OCR to it
    matcher1 = re.search(pattern1,path)
    new_path = "{}_black_org.png".format(matcher1.group(0))
    cv2.imwrite(new_path, gray)
    return gray
def readImgBin(path):
    image = cv2.imread(path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
 
    # check to see if we should apply thresholding to preprocess the
    # image
#     if "thresh" == "thresh":
    gray = cv2.threshold(gray, 180, 255,cv2.THRESH_BINARY)[1]
    # noise
#     elif "blur" == "blur":
#     gray = cv2.medianBlur(gray, 3)
    # write the grayscale image to disk as a temporary file so we can
    # apply OCR to it
    matcher1 = re.search(pattern1,path)
    new_path = "{}_black_bin.png".format(matcher1.group(0))
    cv2.imwrite(new_path, gray)
    return gray

In [478]:
def edgeDetect(path):
    p1 = r"(.*[0-9])"
    pattern1 = re.compile(p1)
    image = cv2.imread(path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gaus = cv2.GaussianBlur(gray,(11,11),0)
    edges = cv2.Canny(gaus,50,50, 3)   
    matcher1 = re.search(pattern1,path)
    new_path = "{}_edge.png".format(matcher1.group(0))
    cv2.imwrite(new_path, edges)
import glob   
# path = 'testset/*_black_org.png' 
path = 'testset/*.jpg'
files=glob.glob(path)   
for file in files:  
#     readImgOrg(file)
    edgeDetect(file)
#     readImgBin(file)

In [476]:
def lineDetect(path):
    p1 = r"(.*[0-9])"
    pattern1 = re.compile(p1)
    image = cv2.imread(path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gaus = cv2.GaussianBlur(gray,(11,11),0)
    edges = cv2.Canny(gaus,50,50, 3)  
    
    lines = cv2.HoughLinesP(edges,1,np.pi * 10/180,1,10000,minLineLength=10, maxLineGap=5)
#     lines = cv2.HoughLines(edges,1,np.pi * 10/180, 200)
#     if(lines is not None):
#         for l in lines:
#             for rho,theta in l:
#                 a = np.cos(theta)
#                 b = np.sin(theta)
#                 x0 = a*rho
#                 y0 = b*rho
#                 x1 = int(x0 + 1000*(-b))
#                 y1 = int(y0 + 1000*(a))
#                 x2 = int(x0 - 1000*(-b))
#                 y2 = int(y0 - 1000*(a))
#                 cv2.line(image,(x1,y1),(x2,y2),(0,0,255),2)
    ll = set()
    if(lines is not None):
        for l in lines:
            for x1,y1,x2,y2 in l:
                ll.add((x1,y1,x2,y2))
    for l in ll:
        cv2.line(image,(l[0],l[1]),(l[2],l[3]),(0,255,0),3, 8)

    matcher1 = re.search(pattern1,path)
    new_path = "{}_edge_line_gaus.png".format(matcher1.group(0))
    cv2.imwrite(new_path, image)
    
    

In [477]:
import glob   
# path = 'testset/*_black_org.png' 
path = 'testset/*.jpg'
files=glob.glob(path)   
for file in files:  
#     readImgOrg(file)
    lineDetect(file)
#     readImgBin(file)

In [447]:
gray = readImg("testset/2.jpg")

In [275]:
# load the image as a PIL/Pillow image, apply OCR, and then delete
# the temporary file
img = cv2.imread("testset/2_black.png")
h, w, _ = img.shape
boxes = pytesseract.image_to_boxes(img)
text = pytesseract.image_to_string(img)

# for (a,b) in (boxes.splitlines():
#     b = b.split(' ')
# #     img = cv2.rectangle(img, (int(b[1]), h - int(b[2])), (int(b[3]), h - int(b[4])), (0, 255, 0), 2)
#     print int(b[3]) - int(b[1]) 

In [152]:
def get_card_id(text):
    card_id = ""
    for cor in text.split():
        p1 = r"[0-9X]+"
        pattern1 = re.compile(p1)
        matcher1 = re.search(pattern1,cor)
        if(matcher1 != None):
            ss = matcher1.group(0)
            if(len(ss) >= 10):
                card_id = ss
    return card_id
    

In [117]:
vv = set()

In [118]:
text = pytesseract.image_to_string("testset/19_black.png",  config="-c tessedit_char_whitelist=0123456789X -psm 6")

In [119]:
print text

113 45  5133
73 5 9   3 5 52
122 199251151188  3 
12   5  5  3    
 111 1  
3   1   451252 3303291992111824510


In [264]:
import glob   
path = 'testset/*_black_org.png'   
files=glob.glob(path)   
p1 = r"[0-9]+"
cc = 0
for file in files:  
    img = cv2.imread(file)
    h, w, _ = img.shape
    text = pytesseract.image_to_string(img, config="-c tessedit_char_whitelist=0123456789X -psm 6")
    
    print file
    aa = get_card_id(text)
    if(aa != ""):
        pattern1 = re.compile(p1)
        matcher1 = re.search(pattern1,file)
        id_ = matcher1.group(0)
        vv.add(id_)
        cc += 1
    print aa
print (len(vv) + 0.0)/ len(files)
print len(files)
print cc

testset/0_black_org.png
19321251332
testset/100_black_org.png
31933963731313
testset/10_black_org.png
359559535535
testset/11_black_org.png
350784198610203334
testset/12_black_org.png
3333331333323
testset/13_black_org.png
9514157313213511353233731121332331153
testset/14_black_org.png
620421199008092035
testset/17_black_org.png
47131733274547
testset/19_black_org.png
330329199211182454
testset/1_black_org.png
3135078319820520301X
testset/20_black_org.png
510131198601051521
testset/21_black_org.png
530127198710073227
testset/22_black_org.png
9533133136
testset/23_black_org.png
441421198704282417
testset/24_black_org.png
3335555555553515535
testset/25_black_org.png
0131020233523203175
testset/28_black_org.png
558312511535539405529
testset/29_black_org.png
510722198707305952
testset/2_black_org.png
620103197808042310
testset/30_black_org.png
13325233223273212
testset/33_black_org.png
65222216615
testset/34_black_org.png
22322333233131322333
testset/35_black_org.png
43951453553335
testset/

In [63]:
get_card_id(text)

u'620103197808042310'

In [ ]:
#image segmentation

In [ ]:
def edgeDetect(path):
    p1 = r"(.*[0-9])"
    pattern1 = re.compile(p1)
    image = cv2.imread(path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gaus = cv2.GaussianBlur(gray,(11,11),0)
    edges = cv2.Canny(gaus,50,50, 3)   
    matcher1 = re.search(pattern1,path)
    new_path = "{}_edge.png".format(matcher1.group(0))
    cv2.imwrite(new_path, edges)

In [487]:
def getLaplacianImg(path):
    p1 = r"(.*[0-9])"
    pattern1 = re.compile(p1)
    image = cv2.imread(path)
    # Create a kernel that we will use to sharpen our image
    # an approximation of second derivative, a quite strong kernel
    kernel = np.array([[1, 1, 1], [1, -8, 1], [1, 1, 1]], dtype=np.float32)
    # do the laplacian filtering as it is
    # well, we need to convert everything in something more deeper then CV_8U
    # because the kernel has some negative values,
    # and we can expect in general to have a Laplacian image with negative values
    # BUT a 8bits unsigned int (the one we are working with) can contain values from 0 to 255
    # so the possible negative number will be truncated
    imgLaplacian = cv2.filter2D(image, cv2.CV_32F, kernel)
    sharp = np.float32(image)
    imgResult = sharp - imgLaplacian
    # convert back to 8bits gray scale
    imgResult = np.clip(imgResult, 0, 255)
    imgResult = imgResult.astype('uint8')
    imgLaplacian = np.clip(imgLaplacian, 0, 255)
    imgLaplacian = np.uint8(imgLaplacian)
    matcher1 = re.search(pattern1,path)
    new_path = "{}_lap.png".format(matcher1.group(0))
    cv2.imwrite(new_path, imgResult)

In [488]:
import glob
path = 'testset/*.jpg'
files=glob.glob(path)   
for file in files:  
#     readImgOrg(file)
    getLaplacianImg(file)
#     readImgBin(file)

In [509]:
?cv2.distanceTransform

In [510]:
?cv2.threshold

In [550]:
image = cv2.imread("testset/0_lap.png")
bw = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
_, bw = cv2.threshold(bw, 40, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)

In [551]:
dist = cv2.distanceTransform(bw, cv2.DIST_L2, 3)

In [552]:
cv2.normalize(dist, dist, 0, 1.0, cv2.NORM_MINMAX)
kernel1 = np.ones((3,3), dtype=np.uint8)
dist = cv2.dilate(dist, kernel1)
# Create the CV_8U version of the distance image
# It is needed for findContours()
dist_8u = dist.astype('uint8')
# Find total markers
_, contours, _ = cv2.findContours(dist_8u, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
# Create the marker image for the watershed algorithm
markers = np.zeros(dist.shape, dtype=np.int32)
# Draw the foreground markers
for i in range(len(contours)):
    cv2.drawContours(markers, contours, i, (i+1), -1)
# Draw the background marker
cv2.circle(markers, (5,5), 3, (255,255,255), -1)

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ..., 
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int32)

In [555]:
rr = cv2.normalize(markers, markers, 0, 255, cv2.NORM_MINMAX)

In [526]:
cv2.imshow("Distance Transform Image", dist);

In [560]:
# def getBinaryImgAndDTrans(path):
#     p1 = r"(.*[0-9])"
#     pattern1 = re.compile(p1)
#     image = cv2.imread(path)
#     bw = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
#     _, bw = cv2.threshold(bw, 40, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
#     dist = cv2.distanceTransform(bw, cv2.DIST_L2, 3)
#     cv2.normalize(dist, dist, 0, 1, cv2.NORM_MINMAX)
#     _, dist = cv2.threshold(dist, 0.1, 1.0, cv2.THRESH_BINARY)
#     kernel1 = np.ones((3,3), dtype=np.uint8)
#     dist = cv2.dilate(dist, kernel1)
#     # Create the CV_8U version of the distance image
#     # It is needed for findContours()
#     dist_8u = dist.astype('uint8')
#     # Find total markers
#     _, contours, _ = cv2.findContours(dist_8u, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
#     # Create the marker image for the watershed algorithm
#     markers = np.zeros(dist.shape, dtype=np.int32)
#     # Draw the foreground markers
#     for i in range(len(contours)):
#         cv2.drawContours(markers, contours, i, (i+1), -1)
#     # Draw the background marker
#     cv2.circle(markers, (5,5), 3, (255,255,255), -1)
    
#     matcher1 = re.search(pattern1,path)
#     new_path = "{}_lap_edge.png".format(matcher1.group(0))
#     cv2.imwrite(new_path, bw)
#     trans_path = "{}_dtrans.png".format(matcher1.group(0))
#     cv2.imwrite(trans_path, markers)
# import glob
# path = 'testset/*lap.png'
# files=glob.glob(path)   
# for file in files:  
# #     readImgOrg(file)
#     getBinaryImgAndDTrans(file)
# #     readImgBin(file)

In [579]:
markers = np.zeros(dist.shape, dtype=np.int32)
cv2.rectangle(markers, (450,300),(900,550),(255,255,255), -1)
cv2.imwrite("testset/00.png", markers)

True

In [626]:
dist.shape

(866, 1380)

In [605]:
?cv2.bitwise_not

In [618]:
?cv2.circle

In [684]:
?cv2.threshold

In [687]:
?cv2.threshold

In [696]:
?cv2.threshold

In [727]:
def watershed(path):
    p1 = r"(.*[0-9])"
    pattern1 = re.compile(p1)
    image = cv2.imread(path)
    kernel = np.array([[1, 1, 1], [1, -8, 1], [1, 1, 1]], dtype=np.float32)
    # do the laplacian filtering as it is
    # well, we need to convert everything in something more deeper then CV_8U
    # because the kernel has some negative values,
    # and we can expect in general to have a Laplacian image with negative values
    # BUT a 8bits unsigned int (the one we are working with) can contain values from 0 to 255
    # so the possible negative number will be truncated
#     imgLaplacian = cv2.filter2D(image, cv2.CV_32F, kernel)
#     sharp = np.float32(image)
#     imgResult = sharp - imgLaplacian
    # convert back to 8bits gray scale
    image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    imgResult = np.clip(image, 0, 255)
    imgResult = imgResult.astype('uint8')
    image = cv2.GaussianBlur(image,(11,11),0)
    bw = cv2.cvtColor(imgResult, cv2.COLOR_BGR2GRAY)
    markers = np.zeros(bw.shape, dtype=np.int32)
    h = markers.shape[0]
    w = markers.shape[1]
    cv2.circle(markers, (5,5), 10, (255, 255, 255), -1)
    cv2.circle(markers, (5,h - 5), 10, (255, 255, 255), -1)
    cv2.circle(markers, (w - 5,5), 10, (255, 255, 255), -1)
    cv2.circle(markers, (w - 5,h - 5), 10, (255, 255, 255), -1)
    cv2.rectangle(markers, (int(w * 0.4),int(h * 0.4)),(int(w * 0.6), int(h * 0.6)),(50,50,50), -1)
    # Perform the watershed algorithm
    cv2.watershed(image, markers)
#     mark = markers.astype('uint8')
#     mark = cv2.bitwise_not(mark)
#     # uncomment this if you want to see how the mark
#     # image looks like at that point
#     #cv.imshow('Markers_v2', mark)
#     # Generate random colors
#     colors = []
#     for contour in contours:
#         colors.append((rng.randint(0,256), rng.randint(0,256), rng.randint(0,256)))
#     # Create the result image
#     dst = np.zeros((markers.shape[0], markers.shape[1], 3), dtype=np.uint8)
#     # Fill labeled objects with random colors
#     for i in range(markers.shape[0]):
#         for j in range(markers.shape[1]):
#             index = markers[i,j]
#             if index > 0 and index <= len(contours):
#                 dst[i,j,:] = colors[index-1]
    matcher1 = re.search(pattern1,path)
    new_path = "{}_watershed_hsv.png".format(matcher1.group(0))
#     cv2.normalize(mark, mark, 0, 255, cv2.NORM_MINMAX)
    cv2.imwrite(new_path, markers.astype('uint8'))
    _,markers = cv2.threshold( markers.astype('uint8'),  200, 255, cv2.THRESH_BINARY );
    thresh_callback(markers, path)
import glob
path = 'testset/*.jpg'
files=glob.glob(path)   
for file in files:  
#     readImgOrg(file)
    watershed(file)
#     readImgBin(file)

2
testset/0.jpg
0.00306706405403
2
testset/1.jpg
0.180515704866
2
testset/10.jpg
0.0560238780875
2
testset/100.jpg
0.0259442468065
2
testset/11.jpg
0.00581297315526
2
testset/12.jpg
0.0100940145431
2
testset/13.jpg
0.00395582777353
2
testset/14.jpg
0.00582357113114
2
testset/17.jpg
0.0662009139459
2
testset/19.jpg
0.00675200782523
2
testset/2.jpg
0.00811056424929
2
testset/20.jpg
0.00432432487738
2
testset/21.jpg
0.00687103687567
2
testset/22.jpg
0.0379267304618
2
testset/23.jpg
0.0820666760369
2
testset/24.jpg
0.00945406112816
2
testset/25.jpg
0.0770091687831
2
testset/28.jpg
0.00572084708878
2
testset/29.jpg
0.0151979941182
2
testset/30.jpg
0.0122665239937
2
testset/33.jpg
0.180456215587
2
testset/34.jpg
0.0985886372277
2
testset/35.jpg
0.0128541293385
2
testset/36.jpg
0.0819254946722
2
testset/37.jpg
0.00328537593856
2
testset/38.jpg
0.0861679144077
2
testset/39.jpg
0.199171402249
2
testset/40.jpg
0.0778290476306
2
testset/41.jpg
0.00318763833087
2
testset/42.jpg
0.0617243212172
2
t

In [726]:
def thresh_callback(src_gray, path):
    p1 = r"(.*[0-9])"
    pattern1 = re.compile(p1)
    # Detect edges using Canny
    # Find contours
    _, contours, _ = cv2.findContours(src_gray, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    # Find the convex hull object for each contour
    hull_list = []
    for i in range(len(contours)):
        hull = cv2.convexHull(contours[i])
        hull_list.append(hull)
    # Draw contours + hull results
    drawing = np.zeros((src_gray.shape[0], src_gray.shape[1], 3), dtype=np.uint8)
    print len(contours)
    print path
    for i in range(len(contours)):
        if(i == 0):
            continue
        color = (rng.randint(0,256), rng.randint(0,256), rng.randint(0,256))
#         print contours
#         cv2.drawContours(drawing, contours, i, color)
        cv2.drawContours(drawing, hull_list, i, color)
        
        print (cv2.contourArea(hull_list[i]) - cv2.contourArea(contours[i])) / cv2.contourArea(hull_list[i])
    matcher1 = re.search(pattern1,path)
    new_path = "{}_watershed_convex.png".format(matcher1.group(0))
#     cv2.normalize(mark, mark, 0, 255, cv2.NORM_MINMAX)
    cv2.imwrite(new_path, drawing)